# Autograd

- PyTorch의 `autograd` 패키지는 신경망에서 역전파 단계의 연산을 자동화할 수 있음
    - 신경망의 역전파 단계를 직접 구현할 필요가 없게 해줌
- `x`가 `x.requires_grad=True` 인 Tensor이면 `x.grad`는 어떤 스칼라 값에 대한 x 의 변화도를 갖는 또 다른 Tensor

## PyTorch: Tensor와 autograd

### Example
- Fully Connected Network
    - 1 hidden layer
    - ReLU
- 출력과 정답 사이의 유클리드 거리 (Euclidean distance)를 최소화하도록 Optimize
- 경사하강법(gradient descent) 사용

In [1]:
"""
네트워크 구성
"""

# N : batch size
# H : hidden layer의 차원
# D_in : 입력의 차원
# D_out : 출력의 차원
N, H, D_in, D_out = 64, 100, 1000, 10

# learning rate
lr = 1e-6

In [2]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

x = torch.randn(N, D_in, device=device, dtype=torch.float)
y = torch.randn(N, D_out, device=device, dtype=torch.float)

# gradient를 계산해야하므로, requires_grad=True
w1 = torch.randn(D_in, H, device=device, dtype=torch.float, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=torch.float, requires_grad=True)

for t in range(500):
    # Forward pass
    # 중간의 값들을 가지고 있을 필요가 없음
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Loss
    loss = (y_pred - y).pow(2).sum()
    if t % 10 == 0:
        print(t, loss.item())
    
    # Backprop
    loss.backward()
    
    # Update
    with torch.no_grad():
        # 가중치들에 대해서는 추적할 필요가 없으므로, torch.no_grad()로 감쌌음
        w1 -= lr * w1.grad
        w2 -= lr * w2.grad
        
        # gradient 계산 후, 0으로 비워주기
        w1.grad.zero_()
        w2.grad.zero_()


print("")
print((y_pred - y).sum().item())

0 28300280.0
10 3665564.5
20 231582.796875
30 70265.34375
40 26504.51171875
50 11123.322265625
60 5001.181640625
70 2375.09423828125
80 1181.289794921875
90 607.9930419921875
100 321.8479309082031
110 174.64524841308594
120 96.58863830566406
130 54.264404296875
140 30.888309478759766
150 17.779647827148438
160 10.330820083618164
170 6.051975250244141
180 3.5700149536132812
190 2.1188526153564453
200 1.264204502105713
210 0.7577446699142456
220 0.45608383417129517
230 0.2754395008087158
240 0.16693682968616486
250 0.10146401822566986
260 0.0618303157389164
270 0.037796102464199066
280 0.02321103774011135
290 0.014317645691335201
300 0.008905576542019844
310 0.005594547372311354
320 0.0035744633059948683
330 0.002320762723684311
340 0.001544092781841755
350 0.0010544552933424711
360 0.000740090967155993
370 0.0005362571100704372
380 0.00039672659477218986
390 0.0003005779581144452
400 0.00023131509078666568
410 0.00018284382531419396
420 0.00014644973271060735
430 0.00011903923586942255


## PyTorch: 새 autograd 함수 정의하기

### `autograd`의 기본(primitive) 연산자
- `forward()`
    - 입력 Tensor로부터 출력 Tensor를 계산
- `backward()`
    - 어떤 스칼라 값에 대한 출력 Tensor의 변화도를 전달받고, 동일한 스칼라 값에 대한 입력 Tensor의 변화도를 계산

### 사용자 정의 autograd 연산자
- 사용자 정의 autograd 연산자는 `torch.autograd.Function` 의 서브클래스(subclass)를 정의하고 `forward` 와 `backward` 함수를 구현하여 정의할 수 있음

### Example
ReLU로 비선형적(nonlinearity)으로 동작하는 사용자 정의 autograd 함수를 정의하여 2-계층 신경망에 적용하기

In [3]:
import torch

class MyReLU(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        # ctx : context object를 의미
        # ctx.save_for_backward()로 backward()에서 사용할 정보 저장(cache)
        
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        # ctx.saved_tensors로 forward()에서 저장한 정보를 가져올 수 있음
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

In [4]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

x = torch.randn(N, D_in, device=device, dtype=torch.float)
y = torch.randn(N, D_out, device=device, dtype=torch.float)

w1 = torch.randn(D_in, H, device=device, dtype=torch.float, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=torch.float, requires_grad=True)

for t in range(500):
    # Function.apply를 통해 사용자 정의 Function을 적용
    relu = MyReLU.apply
    
    # Forward pass
    y_pred = relu(x.mm(w1)).mm(w2) # 사용자 정의 Function인 relu 사용
    
    # Loss
    loss = (y_pred - y).pow(2).sum()
    if t % 10 == 0:
        print(t, loss.item())
    
    # Backprop
    loss.backward()
    
    # Update
    with torch.no_grad():
        w1 -= lr * w1.grad
        w2 -= lr * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()


print("")
print((y_pred - y).sum().item())

0 32125400.0
10 1291152.125
20 277882.3125
30 99130.203125
40 41854.14453125
50 19480.7890625
60 9639.337890625
70 4981.47265625
80 2650.34765625
90 1442.2786865234375
100 799.3369140625
110 449.7394714355469
120 256.4328308105469
130 147.93881225585938
140 86.25763702392578
150 50.77599334716797
160 30.15117835998535
170 18.046537399291992
180 10.879755020141602
190 6.602893829345703
200 4.031257629394531
210 2.474745273590088
220 1.5269272327423096
230 0.9463138580322266
240 0.5888106226921082
250 0.36781948804855347
260 0.23052042722702026
270 0.14490440487861633
280 0.09136039018630981
290 0.057742055505514145
300 0.03661506623029709
310 0.02330731973052025
320 0.014900961890816689
330 0.009598630480468273
340 0.0062277596443891525
350 0.004094322212040424
360 0.0027288866695016623
370 0.00185498408973217
380 0.001284814323298633
390 0.0009048372157849371
400 0.0006521072937175632
410 0.0004797900328412652
420 0.00035890701110474765
430 0.0002736023161560297
440 0.00021277306950651